<a href="https://colab.research.google.com/github/YeaAyuni/vlad-sd-webui/blob/main/Vlad_WEBUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🪪 Initialize

In [ ]:
#@title Add Google Drive
from google.colab import drive
MOUNTED_DRIVE_PATH = '/content/drive'
drive.mount(MOUNTED_DRIVE_PATH)

### Requirements

In [ ]:
#@title ##### <font color="orange">Important. </font> _Import Packages, Global-Variabels, Utility Functions_

import os
import subprocess
import shutil
import json
import threading
import time

from IPython.display import clear_output
from urllib.parse import urlparse
from queue import Queue

WORKING_DIRECTORY_NAME = "VLADMANDIC-SD" #@param {type:"string"}
ROOT_DIRECTORY = "/content"
WD_path = f"{ROOT_DIRECTORY}/{WORKING_DIRECTORY_NAME}"

default_MODELS_PATH = f"{WD_path}/models"
default_VAE_PATH = f"{default_MODELS_PATH}/VAE"
default_CHECKPOINT_PATH = f"{default_MODELS_PATH}/Stable-diffusion"
default_REPOSITORIES_PATH = f"{WD_path}/repositories"

# utility functions
def downloadFile(url):
  !wget --content-disposition --no-clobber "$url"


def getRepository(url, dirname="",  hash=""):
  cloneCommand = ['git', 'clone', url]
  if dirname != "":
      cloneCommand.append(dirname)

  subprocess.run(cloneCommand)

  if hash != "" and dirname != ".":
    cwd = os.getcwd()
    reponame = url.split('/')[-1].strip() if dirname == "" else dirname

    os.chdir(f"{cwd}/{reponame}")

    print("Using commit hash.")
    checkoutCommand = ['git', 'checkout', hash]

    subprocess.run(checkoutCommand)

  return os.getcwd()


def splitString(string, spliter=""):
  return [str.strip() for str in string.split(spliter)]


def createTempFolder(foldername, fn, rm_source=True):
  os.chdir(ROOT_DIRECTORY)

  targetDir = f"{ROOT_DIRECTORY}/{foldername}"

  try:
    try:
      os.makedirs(foldername)
    except OSError:
      shutil.rmtree(targetDir, ignore_errors=True)
      os.makedirs(foldername)

    os.chdir(targetDir)
  except OSError as err:
    print(err)

  fn(os.getcwd())

  if rm_source:
    print("Removing Temp")
    shutil.rmtree(targetDir, ignore_errors=True)

  return targetDir


def createFolder(path, foldername="", chdir=False):
  fullpath = path if foldername == "" else os.path.join(path, foldername)
  os.makedirs(fullpath ,exist_ok = True)
  if chdir:
    os.chdir(fullpath)

  return fullpath


def cloneFolder(source, destination, rm_source=True, exclude:list=['.git','.gitattributes']):
  try:
    for item in os.listdir(source):
      if item not in exclude:
        filePath = os.path.join(source, item)

        if os.path.exists(f"{destination}/{item}"):
          print(f"FILE FROM: '{filePath}', ALREADY IN: '{destination}'")
          continue

        shutil.move(filePath, destination)
        print(f"  > {item}")

  except OSError as err:
    print("FOLDER NOT FOUND")

  if rm_source:
    print("Removing Source")
    shutil.rmtree(source, ignore_errors=True)



def useTemplate(tempfoldername, destination, templates:dict, logs):
    """
    templates: {
      "GIT_REPO": "repository url",
    }
    """
    if not os.path.exists(destination):
      createFolder(path=destination)

    def getRepoCallback(currentPath):
      # templates
      for template in templates:
        if template == "GIT_REPO":
          repoUrl = templates[template]
          getRepository(repoUrl, dirname=".")
          logs.createLog(f"Downloaded '{repoUrl}'")

      cloneFolder(source=currentPath, destination=destination)
    createTempFolder(foldername=tempfoldername, fn=getRepoCallback)


def waitFor(fn, max_attempts=10, delay=1):
  attempt=1
  while attempt <= max_attempts:
    try:
      fn()
      break
    except Exception as err:
      print(f"Attempt {attempt} | Waiting target, Retrying...")
      time.sleep(delay)
      attempt = attempt + 1
  else:
    print("Failed To Resolve Target, Max Attempt Reached.")


def copy_file(src, destination):
    file_name = os.path.basename(src)
    destinationPath = os.path.join(destination, file_name)
    shutil.copy(src, destinationPath)


def isUrlOrPath(args):
    if os.path.exists(args):
        return "PATH"

    parsed_url = urlparse(args)
    if parsed_url.scheme and parsed_url.netloc:
        return "URL"

    return None


def verifyUrlOrPath(string:str, fn) -> list:
  urls = splitString(string, spliter=",")

  for url in urls:

    if isUrlOrPath(url) == "PATH":
      with open(url, "r") as file:
        url = ",".join([line.strip() for line in file.readlines() if line != ""])
      file.close()

    if url != "" and isUrlOrPath(url) != None:
      fn(splitString(url, spliter=","))

# Launch server utility
def with_ngrok(auth_token, logs, port=7860):
  try:
    import pyngrok
  except ImportError:
    !pip install pyngrok

  global ngrok
  from pyngrok import ngrok

  import requests

  ngrok_ready = False

  while True:
    try:
      isUiReady = True if requests.get(f'http://127.0.0.1:{port}').status_code == 200 else False

      if isUiReady and not ngrok_ready:

        ngrok.set_auth_token(auth_token)
        public_url = ngrok.connect(7860).public_url
        ngrok_ready = True

        clear_output()
        print(f"NGROK URL: {public_url}")

        break
    except:
      time.sleep(1)
      pass


# Loggings
class Logging():
  def __init__(self):
    self.logs = []

  def createLog(self, message, options:dict={}):
    colorOptions = {
        "black":30,
        "red":31,
        "green": 32,
        "yellow":33,
        "blue":34,
        "purple":35,
        "cyan":36,
        "white":37
    }
    styleOptions = {
        "no-effect":0,
        "bold":1
    }
    backgroundOptions = {
        "black":40,
        "red":41,
        "green": 42,
        "yellow":43,
        "blue":44,
        "purple":45,
        "cyan":46,
        "white":47
    }

    colorCode =  colorOptions[options.get("color") or "green"]
    styleCode = styleOptions[options.get("weight") or "no-effect"]
    backgroundCode= backgroundOptions[options.get('bg') or "black"]

    format = f"\033[{styleCode};{colorCode};{backgroundCode}m" or "\033[32m"

    self.logs.append(f"{format}{message}\033[0m")
    clear_output()
    self.printLogs()

  def printLogs(self):
    for log in self.logs:
      print(log)

In [ ]:
#@title ##### Main Repository

mainRepoLogging = Logging()

def mainRepository():
  custom_commit_hash = "a02c43a90767a30ed37faf6723199a42f4b3deb0" #@param {type:"string"}
  reinstall = True #@param {type:"boolean"}
  # @markdown - leave it empty to use latest

  %cd "$ROOT_DIRECTORY"
  if reinstall and os.path.exists(WD_path):
    shutil.rmtree(WD_path)
    mainRepoLogging.createLog("Reinstalling Main Repository", options={"color":"yellow"})

  if not os.path.exists(WD_path):
    getRepository("https://github.com/vladmandic/automatic", dirname=WORKING_DIRECTORY_NAME, hash=custom_commit_hash)

  clear_output()


try:

  mainRepository()
  mainRepoLogging.createLog("Initialized Main Repository")

except Exception as err:
  Logging().createLog(f"Failed To Initialize: {err}", options={"color":"red"})

In [ ]:
#@title ##### Install required packages, repositories, and initialize environment variabels

requiredLogs = Logging()


def requiredRepositories():
  required_repositories = {
    'stable-diffusion-stability-ai':'https://github.com/Stability-AI/stablediffusion.git',
    'taming-transformers':'https://github.com/CompVis/taming-transformers.git',
    'k-diffusion':'https://github.com/crowsonkb/k-diffusion.git',
    'CodeFormer':'https://github.com/sczhou/CodeFormer.git',
    'BLIP':'https://github.com/salesforce/BLIP.git',
    'MiDaS':'https://github.com/isl-org/MiDaS'
  }

  os.chdir(default_REPOSITORIES_PATH)

  for repository in required_repositories:
    requiredLogs.createLog(f"installing \033[35m{repository}", options={"color":"yellow"})
    try:
      getRepository(required_repositories[repository], dirname=repository)
      requiredLogs.createLog(f"\033[0m> installed repository => \033[34m{repository}")
    except Exception as err:
      print(err)


def requiredPackages():
  print(f"installing Packages...")
  os.chdir(WD_path)

  !pip install wget
  !pip install tqdm
  !pip install pi-heif

  !pip install timm==0.6.13 --upgrade
  !pip install onnxruntime==1.15.1

  !pip install --upgrade huggingface_hub
  # !pip install --upgrade numpy
  # !pip install --upgrade scipy

  !pip install -r requirements.txt

  !pip install git+https://github.com/openai/CLIP.git
  !pip install git+https://github.com/patrickvonplaten/invisible-watermark.git


def useEnv():
  os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

  # memory allocator
  !apt -y install libjemalloc-dev
  os.environ['LD_PRELOAD'] = '/usr/lib/x86_64-linux-gnu/libjemalloc.so.2'


try:
  requiredRepositories()
  requiredLogs.createLog("---------------------------------------", options={"bg":"cyan"})
  requiredLogs.createLog("Installed Required Repository")

  requiredPackages()
  requiredLogs.createLog("Installed Required Packages")

  requiredLogs.createLog("Use Environment")
except Exception as err:
  print(err)

### +Optional. Get Checkpoint, Lora, LyCoris, Vae, Upscaler , Embeddings.
*this is optional, you can also do it inside web-ui later using extension
**[Batch Downloader](https://github.com/etherealxx/batchlinks-webui)**. But if you want to download models first, then here you go.*'

---

- ## **Input Syntax!**
  - make sure you follow the syntax rules.

> ---

> make sure your input ```url``` is direct link to download file.

> use coma <code>,</code> to separate. example: <i>url1, url2, ..urlN</i>

> also, you can pass  <code>.txt</code> file path. example: <i>filename.txt</i>

> combine links with path: <code>path.txt, url1, url2, path2.txt</code>

> makesure links inside <code>.txt</code> file is **Separated By Line**. example :
```
url1
url2
urlN
```


In [ ]:
#@markdown > ### __Get Checkpoint (CKPT)__
#@markdown ---

models_url = "" #@param {type:"string"}

checkpoint_path = default_CHECKPOINT_PATH
checkpointDownloadLogs = Logging()

createFolder(checkpoint_path, chdir=True)

def checkpointHandler(urls):
  for download_url in urls:
    downloadFile(download_url)
    checkpointDownloadLogs.createLog(f'Downloaded : "{download_url}"')


verifyUrlOrPath(models_url, fn=checkpointHandler)

In [ ]:
#@markdown > ### __Get Low-Rank Adaptation (Lora)__
#@markdown ---

lora_url = "" #@param {type:"string"}

lora_path = f"{WD_path}/models/Lora "
loraDownloadLogs = Logging()

createFolder(lora_path, chdir=True)

def checkpointHandler(urls):
  for download_url in urls:
    downloadFile(download_url)
    loraDownloadLogs.createLog(f'Downloaded : "{download_url}"')

verifyUrlOrPath(models_url, fn=checkpointHandler)

In [ ]:
#@markdown > ### __Get LyCORIS__
#@markdown *Lora beYond Conventional methods, Other Rank adaptation Implementations for Stable diffusion*

#@markdown ---

lyco_url = "" #@param {type:"string"}

lyco_path = f"{WD_path}/models/LyCORIS"
lycoDownloadLogs = Logging()

createFolder(lora_path, chdir=True)

def checkpointHandler(urls):
  for download_url in urls:
    downloadFile(download_url)
    lycoDownloadLogs.createLog(f'Downloaded : "{download_url}"')

verifyUrlOrPath(models_url, fn=checkpointHandler)

In [ ]:
#@markdown > ### __Get Variational Autoencoders (VAE)__
#@markdown ---

#@markdown - template (optional)
use_vae_template_too = False #@param {type:"boolean"}

#@markdown ---
#@markdown - custom url (optional)
vae_url = "" #@param {type:"string"}


#@markdown - you can leave this "custom url" input empty and enable template only

vaeDownloadLogs = Logging()
templates = {
   "GIT_REPO": "https://huggingface.co/JCTN/sd-vae-collection"
}
vae_path = default_VAE_PATH

createFolder(vae_path, chdir=True)

def vaeHandler(urls):
  for download_url in urls:
    downloadFile(download_url)
    vaeDownloadLogs.createLog(f'Downloaded : "{download_url}"')

verifyUrlOrPath(vae_url, fn=vaeHandler)


if use_vae_template_too:
  print("Adding Template...")
  useTemplate(destination=vae_path,
              logs=vaeDownloadLogs,
              tempfoldername="__VAE_TEMPLATE_TEMP",
              templates=templates)

In [ ]:
#@markdown > ### __Get Embeddings__
#@markdown ---

#@markdown - template (optional) <small>[view list here..](https://huggingface.co/embed/negative)</small>
use_embeddings_template_too = False #@param {type:"boolean"}

#@markdown ---
#@markdown - custom url (optional)
embeddings_url = "" #@param {type:"string"}
#@markdown - you can leave this "custom url" input empty and enable template only

templates = {
   "GIT_REPO": "https://huggingface.co/embed/negative"
}
embeddingsDownloadLogs = Logging()
embeddings_path = f"{default_MODELS_PATH}/embeddings/"
negative_embeddings_path = f"{embeddings_path}/negative"

createFolder(embeddings_path, chdir=True)

def embeddingsHandler(urls):
  for download_url in urls:
    downloadFile(download_url)
    embeddingsDownloadLogs.createLog(f'Downloaded : "{download_url}"')

verifyUrlOrPath(embeddings_url, fn=embeddingsHandler)


if use_embeddings_template_too:
    print("Adding Template...")
    useTemplate(destination=negative_embeddings_path,
                logs=embeddingsDownloadLogs,
                tempfoldername="__EMBEDDINGS_TEMPLATE_TEMP",
                templates=templates)

In [ ]:
#@markdown > ### __Get Upsacler__
#@markdown ---

#@markdown - template (optional)
use_upscaler_template_too = False #@param {type:"boolean"}

#@markdown ---
#@markdown - custom url (optional)
esrgan_url = "" #@param {type:"string"}
gfpgan_url = "" #@param {type:"string"}

# ill configure later😴
# bsrgan_url = "" #@param {type:"string"}
# realesrgan_url = "" #@param {type:"string"}

#@markdown - you can leave this "custom url" input empty and enable template only

templates = {
   "GIT_REPO": "https://huggingface.co/uwg/upscaler"
}
upscalerDownloadLogs = Logging()

def upscalerModelHandler():
  custom_urls = []
  custom_urls.append({
      'type': 'ESRGAN',
      'url': esrgan_url
  })
  custom_urls.append({
      'type': 'GFPGAN',
      'url': gfpgan_url
  })

  for upscalerType in custom_urls:

    currentUpscalerUrl = upscalerType.get('url')
    currentUpscalerType = upscalerType.get('type')

    def eachTypeModelHandler(urls):
      for download_url in urls:
        print(download_url)
        foldername = createFolder(default_MODELS_PATH, currentUpscalerType)
        os.chdir(foldername)
        downloadFile(download_url)
        upscalerDownloadLogs.createLog(f"Downloaded '{download_url}'")


    if currentUpscalerUrl == "":
      continue

    verifyUrlOrPath(currentUpscalerUrl, fn=eachTypeModelHandler)

upscalerModelHandler()


if use_upscaler_template_too:

    print("Adding Template...")
    useTemplate(destination=default_MODELS_PATH,
              logs=upscalerDownloadLogs,
              tempfoldername="__UPSCALER_TEMPLATE_TEMP",
              templates=templates)

# 🧩 Extensions

In [ ]:
#@markdown ### NoteBook choice Extension (optional)
#@markdown > extension that i <font color="orange">sugested</font>,  maybe you need it too.
#@markdown You can visit list of extensions [here](https://github.com/YeaAyuni/vlad-sd-webui/tree/main/utils/extension).

#@markdown ---

Reinstall_Notebook_Extensions = False #@param {type:'boolean'}

noteBookExtensionLogs = Logging()

extensionPath = f"{WD_path}/extensions"
buitinExtensionPath = f"{WD_path}/extensions-builtin"

def temporalKit():
  print("Temporal Kit")

def controlNetModel(extensionPath):
  # vesion 1.1, SD 1.5
  model_path = f"{extensionPath}/models"
  os.chdir(model_path)

  #@markdown > Use ControlNet Model
  Model = "Depth" #@param ["None", "Canny", "Depth", "Lineart", "MLSD", "Normal", "OpenPose", "Scribble", "Seg", "ip2p", "Shuffle", "Inpaint", "Softedge", "Lineart_Anime", "Tile", "All"]

  model_urls = {
      "Canny": 'https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth',
      "Depth": 'https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth',
      "Lineart": 'https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart.pth',
      "MLSD": 'https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd.pth',
      "Normal": 'https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae.pth',
      "OpenPose": 'https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth',
      "Scribble": 'https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble.pth',
      "Seg": 'https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg.pth',
      "ip2p": 'https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p.pth',
      "Shuffle": 'https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle.pth',
      "Inpaint": 'https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint.pth',
      "Softedge": 'https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth',
      "Lineart_Anime": 'https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime.pth',
      "Tile": 'https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile.pth'
  }

  def downloadModel(urls=[]):
    for url in urls:
      downloadFile(url)
      noteBookExtensionLogs.createLog(f"Downloaded Model: {url}")

  if Model == "All":
    downloadModel([model_urls.get(m) for m in model_urls])
  elif Model == "None":
    print("Model set to none, skipping")
    pass
  else:
    downloadModel([model_urls.get(Model)])


def nb_extensions_callback(currentPath):
  downloadFile('https://raw.githubusercontent.com/YeaAyuni/vlad-sd-webui/main/utils/extension/sd-extensions.txt')

  NOTEBOOK_EXTENSIONS = f"{currentPath}/sd-extensions.txt"

  with open(NOTEBOOK_EXTENSIONS, "r") as nb_extensions:

    for line in nb_extensions:

      ext = splitString(line, "|")

      os.chdir(buitinExtensionPath)
      if len(ext) == 3 : # simple check for extension/builtin-extension
        os.chdir(extensionPath)

      url = ext[0]
      dirname = ext[1]

      dirname_path = f"{os.getcwd()}/{dirname}"

      print(f"{dirname} ==================================================")

      if os.path.exists(dirname_path):
        if Reinstall_Notebook_Extensions or os.listdir(dirname_path) == []:
          shutil.rmtree(dirname_path)

      # using repository because extension come from git repository
      getRepository(url, dirname=dirname)

      if dirname == 'sd-webui-controlnet':
        controlNetModel(f'{dirname_path}')

      noteBookExtensionLogs.createLog(f"Downloaded Extension: {dirname}")

  nb_extensions.close()


folderPath = createTempFolder(foldername="__NOTEBOOK_EXTENSIONS_TEMP", fn=nb_extensions_callback)

In [ ]:
#@markdown ### Your choice extension

extensionPath = f"{WD_path}/extensions"
createFolder(path=extensionPath, chdir=True)
userChoiceExtensionsLogs = Logging()

source_extensions = "" #@param {type:"string"}
#@markdown - syntax is same as **+optional** cell.
def extensionsHandler(urls):
  for url in urls:
    try:
      getRepository(url)
      userChoiceExtensionsLogs.createLog(f"Downloaded extension '{url}'")
    except Exception as err:
      print(err)

verifyUrlOrPath(string=source_extensions, fn=extensionsHandler)

# 🚀 Run WEB-UI

In [ ]:
#@markdown ### Performance tool
from IPython.display import HTML
import threading

os.chdir(WD_path)
%env TF_CPP_MIN_LOG_LEVEL=1

!apt -y update -qq
!wget http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb \
      https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb \
      https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb \
      https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
!apt -y install -qq libunwind8-dev aria2 libcairo2-dev pkg-config python3-dev
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *.deb
clear_output()

os.chdir(default_MODELS_PATH)
!mkdir -p Diffusers


# --Function AntiDissconnect
os.chdir(WD_path)
audio_url = "https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a"
# Function AntiDissconnect in the background
def play_audio(url):
    display(HTML(f'<audio src="{audio_url}" controls autoplay style="display:none"></audio>'))
# Create a separate thread for AntiDissconnect
audio_thread = threading.Thread(target=play_audio, args=(audio_url,))
audio_thread.start()

In [ ]:
#@markdown ### Launch

launchLogs = Logging()
use_custom_config_path = "" #@param {type:"string"}
always_reinstall_config = False #@param {type:"boolean"}

# always_synchronize_config = False #@param {type:"boolean"}
# #@markdown - synchronize between custom config and default config.

#@markdown ---

use_NGROK_token = "" #@param {type:"string"}
#@markdown > <small>Get NGROK token [here](https://dashboard.ngrok.com/get-started/your-authtoken). *(optional, <font color="yellow" >recomended</font>)*</small>

#@markdown > ---

share_gradio_url = True #@param {type:"boolean"}

os.chdir(WD_path)


try:
  useEnv()
  launchLogs.createLog("Using Env")

  falseLogsOptions = {"color":"purple", "bg":"white"}

  # when using custom config
  default_config_path=f"{WD_path}/config.json"

  if always_reinstall_config:
    os.remove(default_config_path)

  if use_custom_config_path != "" and not os.path.exists(default_config_path):
    copy_file(src=use_custom_config_path, destination=WD_path)
    launchLogs.createLog("Using Custom Config")
  else:
    launchLogs.createLog("Using config that already exists", falseLogsOptions)

  # when using ngrok
  if use_NGROK_token != "":
    ngrok_thread = threading.Thread(target=lambda: with_ngrok(auth_token=use_NGROK_token, logs=launchLogs))
    ngrok_thread.start()

    launchLogs.createLog("Using Ngrok")
  else:
    launchLogs.createLog("Launching without Ngrok", falseLogsOptions)

  launch_ARGS = ["--no-hashing","--insecure"]
  launch_ARGS.append("--share") if share_gradio_url else None

  baked_cmd_args = " ".join(launch_ARGS)

  !COMMANDLINE_ARGS="$baked_cmd_args" python launch.py

finally:
  if use_NGROK_token != "":
    ngrok.get_tunnels()
    ngrok.kill()


# 📖 Readme
more information about this notebook.

### Extensions List
notebook choice extensions list
  - [sd-webui-controlnet](https://github.com/Mikubill/sd-webui-controlnet.git)
  - [batchlinks-webui](https://github.com/etherealxx/batchlinks-webui.git)
  - [tag-autocomplete](https://github.com/DominikDoom/a1111-sd-webui-tagcomplete.git)
  - [ultimate-upscaler](https://github.com/Coyote-A/ultimate-upscale-for-automatic1111.git)
  - [sd-webui-segment-anything](https://github.com/continue-revolution/sd-webui-segment-anything.git)
  - [ebsynth_utility](https://github.com/s9roll7/ebsynth_utility.git)
  - [prompt-generator](https://github.com/AUTOMATIC1111/stable-diffusion-webui-promptgen)
  - [image-adjustment](https://github.com/KohakuBlueleaf/a1111-sd-webui-haku-img)
  - [sd-webui-roop](https://github.com/s0md3v/sd-webui-roop)

# Others (optional)

_just some function that sometime i use, in case you need it._

In [ ]:
#@title Extract Frames From Video
#@markdown > usually i use this when i need to create batch img2img.
#@markdown > output in /content/audios. It will open if you click it, or expand files explorer to view it.

# requirements

!pip install yt-dlp
clear_output()
sourceUrlOrPath = "https://youtu.be/823uvXljaHE" #@param {type:"string"}
name = "Deyang" #@param {type:"string"}

#@markdown - ⬇️ If you using url from youtube, format <code>height:(resolution)</code>
#@markdown <br> example: 1080 resolution input is <code>height:1080</code>
resolution = "height:480" #@param {type:"string"}

name_path = f"/content/audios/{name}"
if os.path.exists(name_path):
  shutil.rmtree(name_path)

frames_path = f"{name_path}/frames"
audio_path = f"{name_path}/audio_source"

os.makedirs(audio_path, exist_ok=True)
%cd "$audio_path"
print("\033[33mDownloading... \033[0m")
!yt-dlp -S "$resolution",ext:mp4:m4a --recode mp4 "$sourceUrlOrPath"



os.makedirs(frames_path, exist_ok=True)
%cd "$frames_path"

#@markdown - ⬇️ select time, format <code>HH:MM:SS</code> |  <code>24FormatHour:Minute:Second</code>
start_time="00:01:23" #@param {type:"string"}
end_time="00:01:28" #@param {type:"string"}

print("\033[32m Splitting Each Frame... \033[0m")

audioSource = f"{audio_path}/{os.listdir(audio_path)[0]}"
!ffmpeg -ss "$start_time" -to "$end_time" -i "$audioSource" frame%5d.png


print("Done ✅")

### Development

In [ ]:
#@markdown ### File Watcher Function
#@markdown skip if you dint use **Automatic_Save_Config_To_Drive**

try:
  import watchdog
except ImportError:
  !pip install imagehash==4.3.1
  !pip install asyncer==0.0.2
  !pip install watchdog

clear_output()
print("Initialized FileWatcherFunction")


from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

class FileUpdateHandler(FileSystemEventHandler):
    def __init__(self,source, destination):
        self.destination = destination
        self.src_folder = source

    def on_modified(self, event):
        if event.is_directory:
            return

        self.copy_file(event.src_path)

    def copy_file(self, src):
      file_name = os.path.basename(src)
      destinationPath = os.path.join(self.destination, file_name)
      shutil.copy(src, destinationPath)


class Watcher:
    def __init__(self, source, destination):
      self.destination = destination
      self.source = source


    def observe(self):
      event_handler = FileUpdateHandler(source=self.source, destination=self.destination)
      observer = Observer()
      observer.schedule(event_handler, path=self.source, recursive=False)

      return observer

def autoSaveConfig(src, dst):
  global config_observePoint

  configWatcher= Watcher(src, dst)
  config_observePoint = configWatcher.observe()

  while not os.path.exists(src):
    time.sleep(10)

  config_observePoint.start()
  print("Config Observe Is Running at ", config_observePoint)


Initialized FileWatcherFunction


In [ ]:
#@markdown ### Using Config File
#@markdown this cell allow you to use configuration before running WEB-UI.
#@markdown What is config? well its config.json file that handle settings like output path, lora path, theme, etc.
#@markdown You can select **_Use Sugested Config_**, or use your own later in <b>Setting</b> Tab inside WEB-UI.

#@markdown <font color="salmon">if you select all, It will use sugested config</font>.

#@markdown ---
Use_Sugested_Config = False #@param {type:'boolean'}
#@markdown - use my configuration file. If Disable it will use default or saved config.json file in your drive if it exists.
#@markdown > ---
Automatic_Save_Config_To_Drive= True #@param {type:'boolean'}
#@markdown - Required drive to mounted. Allow you to save configuration file to google drive, so when you start later it will use latest configuration in your drive only if **_Use_Sugested_Config_** <font color="orange">disabled</font>.

#@markdown ---
#@markdown **TIP!** if you want to use my configuration, then save it to your drive. you can run this cell with  **_Use_Sugested_Config_** enable first, then run it again with  **_Use_Sugested_Config_**  disabled and  **_Automatic_Save_Config_To_Drive_** enabled.


configLogs = Logging()
os.chdir(WD_path)

config_path = f"{WD_path}/config.json"

def verifySugestedConfig():

  configLogs.createLog("Verifying Sugested Config", options={"color":"yellow"})

  checkpoint_path = default_CHECKPOINT_PATH
  vae_path = default_VAE_PATH

  custom_data_url = "https://raw.githubusercontent.com/YeaAyuni/sd-requirements/main/my-customize.json"

  downloadFile(custom_data_url)

  def configValueCheck(path, url, name):
    filepath = f"{path}/{name}"

    if not os.path.isfile(filepath):
      createFolder(path, chdir=True)
      configLogs.createLog(f"\033[31m{name} not exist. \033[33mDownloading")
      downloadFile(url)
      configLogs.createLog(f"Downloaded  \033[34m{name}")

  try:
    with open(config_path) as _config:
      config = json.load(_config)

      checkpointName = config.get("sd_model_checkpoint")
      vaeName = config.get("sd_vae")

      with open('my-customize.json') as _custom:
        custom = json.load(_custom)

        modelDirectUrl = custom.get("checkpoint").get("direct_url")
        vaeDirectUrl = custom.get("vae").get("direct_url")

        configValueCheck(checkpoint_path, modelDirectUrl, checkpointName)
        configValueCheck(vae_path, vaeDirectUrl, vaeName)

      _custom.close()
    _config.close()

    configLogs.createLog("Sugested Config is Verified👍")

  except Exception as err:
    print(err)


def useConfigs(observe=False):
  config_url = "https://raw.githubusercontent.com/YeaAyuni/sd-requirements/main/config.json"

  if Use_Sugested_Config:
    configLogs.createLog("Using Sugested Config")

    if os.path.exists(config_path):
      os.remove(config_path)
    downloadFile(config_url)

    try:
      verifySugestedConfig()
    except Exception as err:
      print("Failed To Use Sugested Config..", err)

  if Automatic_Save_Config_To_Drive and not Use_Sugested_Config:
    global config_thread

    configLogs.createLog("Using Auto Save Config")

    destination_path = f"{MOUNTED_DRIVE_PATH}/MyDrive/SD-WEBUI"
    destination_config_file = f"{destination_path}/{os.path.basename(config_path)}"

    if not os.path.exists(destination_config_file):
      copy_file(src=config_path, destination=destination_path)
    if not os.path.exists(config_path):
      copy_file(src=destination_config_file, destination=WD_path)

    createFolder(destination_path)
    config_thread = threading.Thread(target=autoSaveConfig, args=(config_path, destination_path))

    print("Config thread, running on other thread")
    config_observePoint.start()

useConfigs() # validating run